<a href="https://colab.research.google.com/github/Kazi-Rakib-Hasan-Jawwad/MedMNIST_DL_Benchmark/blob/path_mnist/Simclr_rn18_pathmnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import medmnist
from medmnist import INFO
import torch
from torch import optim, nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
import pytorch_lightning as pl
from torchvision.transforms import transforms
from torchvision.models import mobilenet_v3_small, resnet18
from collections import OrderedDict
import medmnist
from medmnist import INFO
from argparse import Namespace
from pytorch_lightning.loggers import TensorBoardLogger

In [ ]:
class ContrastiveTransformations(object):

    def __init__(self, base_transforms, n_views=2):
        self.base_transforms = base_transforms
        self.n_views = n_views

    def __call__(self, x):
        return [self.base_transforms(x) for i in range(self.n_views)]

In [ ]:
class BatchNorm1dNoBias(nn.BatchNorm1d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bias.requires_grad = False

In [ ]:
class SimCLR2(pl.LightningModule):
    def __init__(self, hidden_dim, lr, temperature, weight_decay, max_epochs, arch='mobilenet_v3_small', data='imagenet'):
        super().__init__()
        self.save_hyperparameters()
        assert self.hparams.temperature > 0.0, 'The temperature must be a positive float!'

        if self.hparams.arch == 'mobilenet_v3_small':
            self.convnet = mobilenet_v3_small(pretrained=False)
            self.encoder_dim = 576
        elif self.hparams.arch == 'resnet18':
            self.convnet = resnet18(pretrained=False)
            self.encoder_dim = 512
        else:
            raise NotImplementedError(f"Architecture {self.hparams.arch} not supported")

        # Remove the final classifier layer
        if self.hparams.arch == 'mobilenet_v3_small':
            self.convnet.classifier = nn.Identity()
        elif self.hparams.arch == 'resnet18':
            self.convnet.fc = nn.Identity()

        # Define the projection head
        self.proj_dim = 128
        projection_layers = [
            ('fc1', nn.Linear(self.encoder_dim, self.encoder_dim, bias=False)),
            ('bn1', nn.BatchNorm1d(self.encoder_dim)),
            ('relu1', nn.ReLU()),
            ('fc2', nn.Linear(self.encoder_dim, self.proj_dim, bias=False)),
            ('bn2', BatchNorm1dNoBias(self.proj_dim)),
        ]
        self.projection = nn.Sequential(OrderedDict(projection_layers))

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(),
                                      lr=self.hparams.lr,
                                      weight_decay=self.hparams.weight_decay)
        lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                                  T_max=self.hparams.max_epochs,
                                                                  eta_min=self.hparams.lr / 50)
        return [optimizer], [lr_scheduler]

    def info_nce_loss(self, batch, mode='train'):
        imgs, _ = batch
        imgs = torch.cat(imgs, dim=0)

        # Encode all images
        feats = self.convnet(imgs)
        proj_feats = self.projection(feats)

        # Calculate cosine similarity
        cos_sim = F.cosine_similarity(proj_feats[:, None, :], proj_feats[None, :, :], dim=-1)

        # Mask out cosine similarity to itself
        self_mask = torch.eye(cos_sim.shape[0], dtype=torch.bool, device=cos_sim.device)
        cos_sim.masked_fill_(self_mask, -9e15)

        # Find positive example -> batch_size//2 away from the original example
        pos_mask = self_mask.roll(shifts=cos_sim.shape[0] // 2, dims=0)

        # InfoNCE loss
        cos_sim = cos_sim / self.hparams.temperature
        nll = -cos_sim[pos_mask] + torch.logsumexp(cos_sim, dim=-1)
        nll = nll.mean()

        # Logging loss
        self.log(f'{mode}_loss', nll)

        # Get ranking position of positive example
        comb_sim = torch.cat([cos_sim[pos_mask][:, None],  # First position positive example
                              cos_sim.masked_fill(pos_mask, -9e15)],
                             dim=-1)
        sim_argsort = comb_sim.argsort(dim=-1, descending=True).argmin(dim=-1)

        # Logging ranking metrics
        self.log(f'{mode}_acc_top1', (sim_argsort == 0).float().mean())
        self.log(f'{mode}_acc_top5', (sim_argsort < 5).float().mean())
        self.log(f'{mode}_acc_mean_pos', 1 + sim_argsort.float().mean())
        return nll

    def training_step(self, batch, batch_idx):
        loss = self.info_nce_loss(batch, mode='train')

        # Log learning rate
        optimizer = self.optimizers()
        current_lr = optimizer.param_groups[0]['lr']
        self.log('lr', current_lr, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        return self.info_nce_loss(batch, mode='val')

In [ ]:
from torchvision.transforms import InterpolationMode
# Initialize the contrastive transformations
contrast_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size=64, interpolation=InterpolationMode.BICUBIC),
    transforms.RandomApply([
        transforms.ColorJitter(
            brightness=0.5,
            contrast=0.5,
            saturation=0.5,
            hue=0.1
        )
    ], p=0.8),
    transforms.RandomGrayscale(p=0.2),
    transforms.GaussianBlur(kernel_size=9),
    transforms.ToTensor()
    #transforms.Normalize((0.5,), (0.5,))
])
c_transform = ContrastiveTransformations(base_transforms=contrast_transforms)

In [ ]:
# Data Loading
num_workers = 6
batch_size = 512
download = True

info = INFO['pathmnist']
root = "/home/rakib/medical_data/"  # Linux
# root = "C:/Users/rakib/medmnist/"  # Windows
DataClass = getattr(medmnist, info['python_class'])

# Load the data
train_dataset = DataClass(split='train', transform=ContrastiveTransformations(contrast_transforms, n_views=2), download=download, size=64, root=root)
val_dataset = DataClass(split='val', transform=ContrastiveTransformations(contrast_transforms, n_views=2), download=download, size=64, root=root)
test_dataset = DataClass(split='test', transform=contrast_transforms, download=download, size=64, root=root)

# Encapsulate data into DataLoader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)


Using downloaded and verified file: /home/rakib/medical_data/pathmnist_64.npz
Using downloaded and verified file: /home/rakib/medical_data/pathmnist_64.npz
Using downloaded and verified file: /home/rakib/medical_data/pathmnist_64.npz


In [ ]:
# Trainer
def train_simclr(max_epochs, encoder_type, train_loader, val_loader, **kwargs):
    trainer = pl.Trainer(
        default_root_dir=f"pretrain_checkpoints/SimCLR_{encoder_type}",
        accelerator="gpu",
        devices=1,
        max_epochs=max_epochs,
        callbacks=[
            ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc_top5'),
            LearningRateMonitor('epoch')
        ],
        log_every_n_steps=1,  # Log every 1 steps
        logger=TensorBoardLogger(save_dir="logs"),  # Log to TensorBoard
    )
    pl.seed_everything(42)  # For reproducibility
    # Define the SimCLR model
    model = SimCLR2(max_epochs=max_epochs, **kwargs)
    trainer.fit(model, train_loader, val_loader)
    # Load the best checkpoint after training
    model = SimCLR2.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    return model

In [ ]:
# Train the model
model = train_simclr(
    max_epochs=100,
    encoder_type='resnet18',
    train_loader=train_loader,
    val_loader=val_loader,
    hidden_dim=128,
    lr=1e-3,
    temperature=0.5,
    weight_decay=1e-6,
    arch='resnet18'
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42
/home/rakib/project-6/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rakib/project-6/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
You are using a CUDA device ('NVIDIA GeForce RTX 3080 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_fl

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [ ]:
print(model)

SimCLR2(
  (convnet): MobileNetV3(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
            (activation): ReLU()
            (scale_activation): Hardsigmoid()
          )
          (2): Conv2dNo

In [ ]:
import copy
network = copy.deepcopy(model.convnet)
network.projection = nn.Identity() # Removing projection head g(.)
print(network)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
class LogisticRegression(pl.LightningModule):

    def __init__(self, lr, weight_decay, max_epochs=100):
        super().__init__()
        self.save_hyperparameters()
        # Mapping from representation h to classes
        self.model = nn.Linear(512, 9)

    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(),
                                lr=self.hparams.lr,
                                weight_decay=self.hparams.weight_decay)
        lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                                      milestones=[int(self.hparams.max_epochs*0.5),
                                                                  int(self.hparams.max_epochs*0.8)],
                                                      gamma=0.1)
        return [optimizer], [lr_scheduler]

    def _calculate_loss(self, batch, mode='train'):
        feats, labels = batch
        preds = self.model(feats)
        loss = F.cross_entropy(preds, labels)
        acc = (preds.argmax(dim=-1) == labels).float().mean()

        self.log(mode + '_loss', loss)
        self.log(mode + '_acc', acc)
        return loss

    def training_step(self, batch, batch_idx):
        return self._calculate_loss(batch, mode='train')

    def validation_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='val')

    def test_step(self, batch, batch_idx):
        self._calculate_loss(batch, mode='test')

In [ ]:
# Prepare data for logistic regression
logistic_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor()
])
# Load the data
logistic_train_dataset = DataClass(split='train', transform=logistic_transform, download=download, size=64, root=root)

logistic_test_dataset = DataClass(split='test', transform=logistic_transform, download=download, size=64, root=root)

# Encapsulate data into DataLoader form
logistic_train_loader = data.DataLoader(dataset=logistic_train_dataset, batch_size=512, shuffle=True, num_workers=4)
#val_loader = data.DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
logistic_test_loader = data.DataLoader(dataset=logistic_test_dataset, batch_size=1024, shuffle=False, num_workers=4)


Using downloaded and verified file: /home/rakib/medical_data/pathmnist_64.npz
Using downloaded and verified file: /home/rakib/medical_data/pathmnist_64.npz


In [ ]:
def prepare_data_features(network, dataloader):
    network.eval()
    network.to(device)
    feats, labels = [], []
    for batch_imgs, batch_labels in tqdm(dataloader):
        batch_imgs = batch_imgs.to(device)
        batch_feats = network(batch_imgs)
        feats.append(batch_feats.detach().cpu())
        labels.append(batch_labels)
    feats = torch.cat(feats, dim=0)
    labels = torch.cat(labels, dim=0)
    # Remove extra dimension
    feats = feats.view(feats.size(0), -1)
    labels = labels.view(-1)
    # Sort images by labels
    labels, idxs = labels.sort()
    feats = feats[idxs]
    return data.TensorDataset(feats, labels)

In [ ]:
from tqdm import tqdm

In [ ]:
device = torch.device('cuda')
train_feats_simclr = prepare_data_features(network, logistic_train_loader)
test_feats_simclr = prepare_data_features(network, logistic_test_loader)
print(train_feats_simclr.tensors[0].shape) # should be torch.Size([sample_number, feat_dimension])
print(train_feats_simclr.tensors[1].shape) # should be torch.Size([sample_number])

100%|█████████████████████████████████████████████| 8/8 [00:00<00:00, 12.36it/s]


torch.Size([89996, 512])
torch.Size([89996])


In [ ]:
# Train logistic regression model
def train_logreg(batch_size, train_feats_data, test_feats_data, model_suffix, max_epochs=100, lr=1e-3, weight_decay=5e-4):
    trainer = pl.Trainer(
        default_root_dir=f"pretrain_checkpoints/SimCLR_mobilenet_{model_suffix}",
        accelerator="gpu",
        devices=1,
        max_epochs=max_epochs,
        callbacks=[
            ModelCheckpoint(save_weights_only=True, mode='max', monitor='val_acc'),
            LearningRateMonitor("epoch")
        ],
        enable_progress_bar=True,
        check_val_every_n_epoch=1
    )
    trainer.logger._default_hp_metric = None
    # Data loaders
    train_loader = data.DataLoader(train_feats_data, batch_size=batch_size, shuffle=True,
                                   drop_last=False, pin_memory=True, num_workers=4)
    test_loader = data.DataLoader(test_feats_data, batch_size=batch_size, shuffle=False,
                                  drop_last=False, pin_memory=True, num_workers=4)
    pl.seed_everything(42)  # To be reproducible
    model = LogisticRegression(lr=lr, weight_decay=weight_decay)
    trainer.fit(model, train_loader, test_loader)
    model = LogisticRegression.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
    # Test best model on train and validation set
    train_result = trainer.test(model, train_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"train": train_result[0]["test_acc"], "test": test_result[0]["test_acc"]}
    return model, result

In [ ]:
# Parameters
batch_size = 512
model_suffix = "experiment_2"  # Update this as needed

# Train and evaluate the model
logreg_model, logreg_result = train_logreg(
    batch_size=batch_size,
    train_feats_data=train_feats_simclr,
    test_feats_data=test_feats_simclr,
    model_suffix=model_suffix,
    max_epochs=100,
    lr=1e-3,
    weight_decay=1e-4
)

# Print results
print(f"Training Accuracy: {logreg_result['train']:.2f}%")
print(f"Testing Accuracy: {logreg_result['test']:.2f}%")

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Seed set to 42
Missing logger folder: pretrain_checkpoints/SimCLR_mobilenet_experiment_2/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type   | Params
---------------------------------
0 | model | Linear | 4.6 K 
---------------------------------
4.6 K     Trainable params
0         Non-trainable params
4.6 K     Total params
0.018     Total estimated model params size (MB)


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=100` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/rakib/project-6/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.


Testing: |                                                | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |                                                | 0/? [00:00<?, ?it/s]

Training Accuracy: 0.95%
Testing Accuracy: 0.92%
